# Data Ingestion

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
# Txt Reader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("example.txt")
text_docs = loader.load()
text_docs

[Document(metadata={'source': 'example.txt'}, page_content='Hola mi nombre es Pedro, soy un Picapiedra.\n\nMe gusta comer hamburguesa y jugar al fútbol.')]

In [10]:
# PDF Reader
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("example_2.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'type': '/Metadata', 'subtype': '/XML', 'source': 'example_2.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Usuarios\n*UsuarioID\nUsuarioNombre\nUsuarioEmail\nUsuarioRol\nUsuarioStatus\nProductos\n*ProductoID\nProductoNombre\nProductoPrecio\nProductoStockMinimo\nProductoFoto\nUnidadMedidaID\nProductoStatus\nClientes\n*ClienteID\nClienteNombre\nClienteDireccion\nClienteTelefono\nClienteStatus\nProveedores\n*ProveedorID\nProveedorNombre\nProveedorCategoria\nProveedorTelefono\nProveedorStatus\nUnidadesMedida\n*UnidadMedidaID\nUnidadMedidaNombre\nUnidadMedidaStatus\nVentas\n*VentaID\nVentaFolio\nVentaFecha\nVentaHora\nClienteID\nUsuarioID\nVentaMetodoPago\nVentaStatus\nVentaDetalle\n*VentaDetalleID\nVentaID\nProductoID\nVentaDetalleCantidad\nVentaDetallePrecio\nVentaDetalleImporte\nEntradaMaterial\n \n*EntradaMaterialID\nEntradaMaterialFolio\nEntradaMaterialFecha\nEntradaMaterialHora\nUsu

In [ ]:
# Web Reader con Beautiful Soup para WebScrapping
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

# Transform

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=50,chunk_overlap=5)
documents=text_splitter.split_documents(text_docs)
documents

[Document(metadata={'source': 'example.txt'}, page_content='Hola mi nombre es Pedro, soy un Picapiedra.'),
 Document(metadata={'source': 'example.txt'}, page_content='Me gusta comer hamburguesa y jugar al fútbol.')]

In [37]:
# Vector Embedding And Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings(model="text-embedding-3-small"))

In [38]:
query = "¿Qué le gusta comer a Pedro?"
retireved_results = db.similarity_search(query)
print(retireved_results[0].page_content)

Me gusta comer hamburguesa y jugar al fútbol.


In [40]:
# OpenAI ChatGPT LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [45]:
# Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Responde la siguiente preguna basado solamente en el contexto dado.
Piensa paso a paso antes de entregar una respuesta detallada.
<context>
{context}
</context>
Question: {input}""")

In [ ]:
# Chain Introduction
# Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)

In [42]:
from langchain.chains import create_retrieval_chain
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [46]:
response = retrieval_chain.invoke({"input":"Me gustaría saber que comida le gusta a Pedro"})

In [ ]:
response["answer"]

'Pedro menciona que le gusta comer hamburguesa en el texto proporcionado. Por lo tanto, la comida que le gusta a Pedro es la hamburguesa.'

In [44]:
# Creación de Calculadora Científica como Tool
from langchain.tools import Tool
from langchain.tools import StructuredTool
import sympy as sp

def scientific_calculator(expression: str) -> str:
    """
    Evalúa una expresión matemática usando SymPy.
    Soporta operaciones científicas avanzadas.
    """
    try:
        result = sp.sympify(expression)
        return str(result)
    except Exception as e:
        return f"Error en el cálculo: {str(e)}"
    
tool = StructuredTool.from_function(
    name="scientific_calculator",
    description="Realiza cálculos científicos avanzados. Usa expresiones matemáticas válidas de Python/SymPy.",
    func=scientific_calculator
)

# Definir la herramienta de calculadora científica
tools = [tool]

# Agents

In [14]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.3)

In [24]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Actúa como matemático experto con 20 años de experiencia"),
        ("user", "Question:{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

In [45]:
# Create Agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [46]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanM

In [47]:
agent_executor.invoke({"input":"¿Cuál es la derivada de: y=(x^2)+(2x)+3 ?"})



> Entering new AgentExecutor chain...

Invoking: `scientific_calculator` with `{'expression': 'diff(x**2 + 2*x + 3, x)'}`


2*x + 2La derivada de \( y = x^2 + 2x + 3 \) es \( 2x + 2 \).

> Finished chain.


{'input': '¿Cuál es la derivada de: y=(x^2)+(2x)+3 ?',
 'output': 'La derivada de \\( y = x^2 + 2x + 3 \\) es \\( 2x + 2 \\).'}